# Jax 101

Здесь представлен черновик для начала работы с библиотекой
[`jax`](https://jax.readthedocs.io/en/latest/index.html).

* Предназначена для работы с многомерными массивами.
* API [`jax.numpy`](https://jax.readthedocs.io/en/latest/jax.numpy.html) соответствует библиотеке [`numpy`](https://numpy.org/).
* API [`jax.scipy`](https://jax.readthedocs.io/en/latest/jax.numpy.html) соответствует библиотеке [`scipy`](https://scipy.org/).
* Реализует *autodiff*, *jit*-компиляцию, различные функции преобразования и многое другое!

In [1]:
running_from_colab = False
if running_from_colab:
    !pip install matplotlib~=3.8.0
    !pip install jax[cpu]~=0.4.19
    !pip install flax~=0.7.4
    !pip install clu~=0.0.10
    !pip install tensorflow>=2.13.0
    !pip install tensorflow_datasets>=4.9.3

## Немного о тензорах

`Array`$-$объект, представляющий собой $k$-мерный массив, $k \geq 0$.

$k = 0$ $-$ это *скаляр* (e.g., коэффициент регуляризации).

In [2]:
import jax
from jax import numpy as jnp

In [3]:
x = jnp.array(2.71828)
x, x.shape

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


(Array(2.71828, dtype=float32, weak_type=True), ())

$k = 1$: *вектор* $\mathbf{y} = [y_1, y_2, \ldots, y_d]^T$, $d \geq 1$
(e.g., параметры перцептрона).

In [4]:
y = jnp.arange(4, dtype=jnp.int32)
y, y.shape

(Array([0, 1, 2, 3], dtype=int32), (4,))

$k = 2$: *матрица* $\mathbf{Z} = (z_{ij})$, $1 \leq i \leq n$, $1 \leq j \leq m$
(e.g., чёрно-белое изображение).

In [5]:
Z = jnp.eye(3)
Z, Z.shape

(Array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], dtype=float32),
 (3, 3))

$k \geq 3$: *тензор* (неформально) (e.g., коллекция чёрно-белых изображений).

In [6]:
T = jnp.arange(24).reshape((2, 3, 4))
T, T.shape

(Array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]], dtype=int32),
 (2, 3, 4))

## Унарные операции

Пусть $f: \mathbb{R} \rightarrow \mathbb{R}$ и $\mathbf{x}$ - $k$-мерный массив.
Тогда $f(\mathbf{x}) = [f(x_i)]$ для каждого элемента $x_i \in \mathbf{x}$.
E.g., применение функций активации.

In [7]:
jnp.exp(jnp.log1p(T))

Array([[[ 1.       ,  2.       ,  3.       ,  4.       ],
        [ 5.       ,  6.       ,  7.0000005,  8.       ],
        [ 9.       , 10.       , 11.000001 , 12.       ]],

       [[12.999999 , 14.000001 , 15.000001 , 16.       ],
        [17.       , 18.       , 19.       , 20.       ],
        [21.000002 , 22.000002 , 23.       , 24.       ]]], dtype=float32)

## Псевдослучайные числа

Первое отличие `jax` от `numpy`: ключ `key` для поглощения случайными функциями
вместо использования глобальной `random.seed`.

In [8]:
key = jax.random.key(0)
shape = (2, 3, 4)
X = jax.random.normal(key, shape)
X

Array([[[ 0.36753944, -0.9082042 , -2.0064416 ,  0.16056262],
        [ 0.13233443, -1.305435  , -0.4055677 , -1.7935358 ],
        [-1.3566552 ,  0.80958456, -0.37977964,  0.08442838]],

       [[-1.895686  , -0.20993415,  0.20252009,  1.3713387 ],
        [-0.60032403, -1.0367845 ,  1.5410699 ,  0.05245331],
        [ 0.03026433,  1.3176132 ,  0.61566246,  1.698919  ]]],      dtype=float32)

In [9]:
X == jax.random.normal(key, shape)

Array([[[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]],

       [[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]]], dtype=bool)

Использование одной и той же `key` возвращает один и тот же результат!

`key` можно разбить на 2 и более ключа и сгенерировать другие значения.

In [10]:
key, subkey = jax.random.split(key)
Y = jax.random.normal(subkey, shape)
Y

Array([[[-2.003785  , -0.25242862,  0.670781  , -0.24748416],
        [-0.8836176 , -0.338196  ,  1.0410497 , -0.47741464],
        [ 0.39571118,  0.838507  , -0.90660936, -0.81707186]],

       [[ 0.43643975,  1.4724442 , -2.19621   , -1.6121409 ],
        [ 1.296409  ,  1.8319693 , -0.9455707 , -0.34352237],
        [ 0.05459858,  0.33989352, -0.74641544, -0.00806977]]],      dtype=float32)

## Бинарные операции

Пусть $\circ: \mathbb{R} \rightarrow \mathbb{R}$, $\mathbf{X}, \mathbf{Y}$
$-$ многомерные массивы одинакового размера.
Тогда $\mathbf{X} \circ \mathbf{Y} = [x_i \circ y_i]$ для всех $i$.
E.g., сложение векторного и позиционного представлений токенов.

In [11]:
X + Y  # X - Y, X * Y, X / Y, X // Y, X ** Y, etc.

Array([[[-1.6362455 , -1.1606328 , -1.3356606 , -0.08692154],
        [-0.75128317, -1.643631  ,  0.635482  , -2.2709506 ],
        [-0.96094406,  1.6480916 , -1.286389  , -0.7326435 ]],

       [[-1.4592463 ,  1.2625101 , -1.9936898 , -0.24080217],
        [ 0.696085  ,  0.79518473,  0.59549916, -0.29106906],
        [ 0.0848629 ,  1.6575067 , -0.13075298,  1.6908493 ]]],      dtype=float32)

## Векторизация

Представление операций на элементах массива без явных *for*-циклов.
Как правило в разы быстрее, чем циклы. См. также предыдущие примеры.

In [12]:
x = jnp.linspace(0, 1, 5_000)
%timeit jnp.array([x[i] * jnp.pi for i in range(x.size)]).block_until_ready()
%timeit (jnp.pi * x).block_until_ready()

1.11 s ± 39.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.32 µs ± 411 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Оси

Строка, столбец, один канал изображений, etc.
E.g., суммирование потерь по оси прецедентов, дропаут по осям прецедентов и токенов, etc.

In [13]:
x = jnp.arange(10).reshape(2, 5)
x, x.sum(axis=0), x.sum(axis=1)

(Array([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]], dtype=int32),
 Array([ 5,  7,  9, 11, 13], dtype=int32),
 Array([10, 35], dtype=int32))

In [14]:
ы = jnp.arange(10).reshape(-1, 10)
ы.sum(axis=0), ы.sum(axis=1)

(Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32), Array([45], dtype=int32))

## Broadcasting

Бинарная операция над массивами в случае, если размеры отличаются:
1. Скопировать элементы одного из или обоих массивов по осям так, чтобы размеры стали одинаковыми.
2. Затем выполнить бинарную операцию.

![Broadcasting](https://numpy.org/doc/stable/_images/broadcasting_4.png)

In [15]:
x = jnp.arange(3).reshape((1, 3))
y = jnp.arange(2).reshape((2, 1))
x, y, x + y, x.T + y.T

(Array([[0, 1, 2]], dtype=int32),
 Array([[0],
        [1]], dtype=int32),
 Array([[0, 1, 2],
        [1, 2, 3]], dtype=int32),
 Array([[0, 1],
        [1, 2],
        [2, 3]], dtype=int32))

In [16]:
x * 3.14, y + 2.71

(Array([[0.  , 3.14, 6.28]], dtype=float32, weak_type=True),
 Array([[2.71],
        [3.71]], dtype=float32, weak_type=True))

In [17]:
X = jnp.arange(24).reshape((2, 3, 4))
Y = jnp.arange(12).reshape((3, 4))
X + Y

Array([[[ 0,  2,  4,  6],
        [ 8, 10, 12, 14],
        [16, 18, 20, 22]],

       [[12, 14, 16, 18],
        [20, 22, 24, 26],
        [28, 30, 32, 34]]], dtype=int32)

## Автоматическое дифференцирование

### Немного про градиентный спуск

Пусть $\ell: \mathbb{R}^d \rightarrow \mathbb{R}_+$ $-$ функция, которую необходимо
минимизировать. Поскольку
$$
f(\mathbf{x} + \boldsymbol{\epsilon}) \approx f(\mathbf{x}) + \boldsymbol{\epsilon}^T \nabla_{\mathbf{x}} f(\mathbf{x}),
$$
где $\nabla_{\mathbf{x}} f(\mathbf{x}) := [\partial_{x_1}f(\mathbf{x}), \ldots, \partial_{x_d}f(\mathbf{x})]^T$ $-$ *градиент* функции $f$,
или направление наискорейшего возрастания функции.
Следовательно при $\boldsymbol{\epsilon} = -\eta \nabla_{\mathbf{x}} f(\mathbf{x}) $
$$
f(\mathbf{x} + \boldsymbol{\epsilon}) \lessapprox f(\mathbf{x})
$$
и обновление параметров
$$
\mathbf{x} \leftarrow \mathbf{x} - \eta \nabla_{\mathbf{x}} f(\mathbf{x})
$$ позволит минимизировать $f$ за несколько итераций.

Таким образом, подсчёт градиента функции - один из важных шагов обучения модели. Но делать это "вручную"
необязательно: *autodiff* (или *autograd*) позволяет строить направленный вычислительный граф с параметрами
функции и выполнять дифференцирование сложной функции автоматически.

In [18]:
x = jnp.arange(4.0)
y = lambda x: jnp.e * jnp.dot(x, x)
jax.grad(y)(x) == 2 * jnp.e * x  # grad(dot(x, x)) = 2x

Array([ True,  True,  True,  True], dtype=bool)

**Вопрос**: что, если $f: \mathbb{R}^d \rightarrow \mathbb{R}^n$ (e.g., потери для каждого из $n$ прецедентов внутри батча)?

**Ответ**: мы, конечно же, можем посчитать *матрицу Якоби* (`jax.jacfwd`, `jax.jacrev`), но наиболее часто нам придётся
считать градиент как раз при подсчёте **суммарной** потери, а суммирование по всем прецедентам даёт скаляр, а не вектор.

In [19]:
x = jnp.arange(4.0)
y = lambda x: x * x
z = lambda x: y(x).sum(axis=0)
jax.jacfwd(y)(x), jax.grad(z)(x)

(Array([[0., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 4., 0.],
        [0., 0., 0., 6.]], dtype=float32),
 Array([0., 2., 4., 6.], dtype=float32))

**Вопрос**: можно ли "убрать" параметр из графа так, чтобы по нему производная не считалась?
(e.g., скрытое состояние рекуррентной сети)

**Ответ**: конечно! Воспользуемся `jax.lax.stop_gradient`:

In [20]:
x = jnp.arange(4.0)
y = lambda x: x * x
u = jax.lax.stop_gradient(y(x))
z = lambda x: u * x
t = lambda x: z(x).sum(axis=0)
jax.grad(t)(x) == y(x)  # grad(detach(y(x)) * x) == y(x)

Array([ True,  True,  True,  True], dtype=bool)

## JIT-компиляция

Компиляция указанной JAX-совместимой функции just-in-time. Декоратор `jax.jit` компилирует
функцию в run-time и сохраняет скомпилированный с помощью [`xla`](https://www.tensorflow.org/xla)
код в кэш, что позволяет впоследствии выполнять функцию быстрее.

In [21]:
def selu(x, alpha=1.67, lambda_=1.05):
    return lambda_ * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

selu_jit = jax.jit(selu)
key = jax.random.key(0)
x = jax.random.normal(key)
selu_jit(x)

%timeit selu(x).block_until_ready()
%timeit selu_jit(x).block_until_ready()

36.2 µs ± 4.26 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
3.98 µs ± 405 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


JIT-компилируемая функция должна быть детерминированной и не должна обладать побочными эффектами, $-$
т.е. должна быть *чистой*. Иначе, результаты могут оказаться непредсказуемыми. Также наличие
условий на входные данные при JIT-компиляции приведут к ошибке компиляции. Для обхода можно указывать
на статические аргументы, а также менять условия на `jax.lax.cond`, если возможно.

## VMAP-векторизация

Автоматическая векторизация функций по указанным осям с помощью `jax.vmap`
(e.g., расширение dot-product attention на несколько прецедентов). Как правило,
работает быстрее, чем for-цикл по каждому прецеденту.

In [22]:
def scaled_dot(x):
    return jnp.dot(x, x) / jnp.sqrt(x.shape[0])

x = jnp.arange(10.0)
scaled_dot(x)

Array(90.12491, dtype=float32)

In [23]:
def batched_scaled_dot(x):  # Без векторизации, с помощью for-цикла.
    return jnp.stack([scaled_dot(x) for x in X])

vmap_scaled_dot = jax.vmap(scaled_dot)  # vmap-векторизация.

X = jnp.arange(2048.0).reshape((-1, 4))  # Батч из векторов размера 4.

%timeit batched_scaled_dot(X).block_until_ready()
%timeit vmap_scaled_dot(X).block_until_ready()

44.3 ms ± 2.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
857 µs ± 39.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
